In [300]:
import requests
import pprint
import pandas as pd
import imaplib
import email

In [301]:
pd.DataFrame(columns=['Date']).to_csv('talenttrove/data/track.csv', index=False)

## Gmail Client

In [4]:
import talenttrove.app.email.gmail 
import importlib
importlib.reload(talenttrove.app.email.gmail)
from talenttrove.app.email.gmail import Gmail
from datetime import datetime,timedelta

In [5]:
username ="agarwalpratham2001@gmail.com"

#generated app password
app_password= "lgjc xmxv ixyr nvxx"
client = Gmail(username, app_password)
client.authenticate()

In [6]:
from_date = (datetime.now() - timedelta(days=1)).strftime("%d-%b-%Y")  # 7 days ago
to_date = (datetime.now()).strftime("%d-%b-%Y")  # Today
ids = client.get_email_by_date(from_date=from_date, to_date=to_date)
data = client.parse_emails(ids)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22/22 [00:07<00:00,  3.00it/s]


In [ ]:
client.close()

## Shortlisting Companies

- https://www.timesbusinessdirectory.com/company-listings?page=9
- https://www.kaggle.com/datasets/peopledatalabssf/free-7-million-company-dataset
- https://beta.data.gov.sg/collections/2/view

In [ ]:
company_df = pd.read_csv("output/companies_sorted.csv")

In [ ]:
company_df=company_df[company_df['industry'] == 'information technology and services']
company_df = company_df[~company_df['size range'].isin(['1 - 10','11 - 50','51 - 200'])]
company_df.drop(columns=['Unnamed: 0','locality']).reset_index(drop=True).to_csv('talenttrove/data/companies/tech_companies.csv')

In [311]:
import importlib
import talenttrove.scrapers.datagov
importlib.reload(talenttrove.scrapers.datagov)
from talenttrove.scrapers.datagov import DataGovCollection

In [312]:
collection = DataGovCollection(2)

In [ ]:
sing_data = collection.collect()

In [ ]:
filter_data = collection.filter(sing_data)

In [ ]:
filter_data.shape

In [ ]:
filter_data.to_csv('talenttrove/data/companies/singapore_companies.csv',index=False)

## Glassdoor

In [ ]:
import http.cookies
cookie_str="""gdsid=1701334685262:1701334685262:7450AAE44F2FF0F0970023DCDB0C952D; asst=1701334685.2; alr=https%3A%2F%2Fwww.google.com%2F; gdId=d68844b8-a3a8-46e6-a377-017bb6d65c4d; __cf_bm=_JUQfgK6vMl1MEvWPuBHYj7Mc6VHlkRGD3iExekwjTk-1701334686-0-ATWNZQexjyGPGaeu4HEll6kosQ25AGx5Tvrn6vbOFleNZg6FMP/b5RVDG0tTjXY/fUDsM/ujnOWRYrIkNB9dIBE=; _cfuvid=CuXEwuj3pRgDtvwrID4LWrRvvZXMr6wl0mk9gWrzrNg-1701334686415-0-604800000; _GD_Job_Session=true; rl_page_init_referrer=RudderEncrypt%3AU2FsdGVkX18zL%2FCncNgcR%2BBqtWtsBcvHAKUiLKrP25Abv3EzitWPkVulSrqNuD54; rl_page_init_referring_domain=RudderEncrypt%3AU2FsdGVkX1%2FGwH0hZFGnjHFxpZyE3K3SNtozE34K2Prn4z5uQ00DuPkyHFH9lctR; GSESSIONID=undefined; indeedCtk=1hgfn65mvjr50801; rsSessionId=1701334686996; _ga=GA1.2.693678502.1701334688; _gid=GA1.2.702267396.1701334688; _optionalConsent=true; AWSALB=L33BDdQqnSkhKn8aZ2HjmHjSKi88ZjG+rXRDvEs0/ULUx7DHaqTsXHU0mI1zTspPpJlP3dwP136i4NWWd32vBiem6r6uDLVvOdl/2YRCaQNBphS+/EGQBTUKwy49; AWSALBCORS=L33BDdQqnSkhKn8aZ2HjmHjSKi88ZjG+rXRDvEs0/ULUx7DHaqTsXHU0mI1zTspPpJlP3dwP136i4NWWd32vBiem6r6uDLVvOdl/2YRCaQNBphS+/EGQBTUKwy49; JSESSIONID=8C28422397E370167EA987047A451059; cass=0; _gcl_au=1.1.654637002.1701334688; _fbp=fb.1.1701334688304.2057378535; g_state={"i_p":1701341890048,"i_l":1}; G_ENABLED_IDPS=google; AFSID=YWU2ZGNkNmQtZGE4Mi00NDc2LWIwYTMtOTVlYTc4MzhkNDVh; at=QAK6Nh09N6FQ2MquounOidap1VH3ubdB6x1eVLstXDTQ8tPYUDgEaJF6MqZb-2-OxiGNrTEdqwJEolHbtdFv18fZ_GIYuMj97jpSsFPHgsVL8rCz6pCbqhot1g5p7AiPtCRsqqHSKPeZWvvCLyDnrgy5dPBpLgwVMMntw9C_O-mOBkRLSkhCg-g2F7-du2yOv2WIS5J0Qiy3l7gbNIM9foZvxnZ8JyM9BrETxZ9q8LUhk_fKI1a3gsqKsds2LPW-0vIPQhYtQcQpVRNMnv7ROAmKGKTRkO-yicAKZCh-hj4PWH6fOEDvbkFMOyM1vBxO4-qjZ-vSrnPVQQk9BOWmM-I1t3mVChPIKCkQaL4D4cQrtanMWS0eS47MbfpA7zx64nDYW0CP7o10jeOZTORVLUEyr-5FlpIs4_EqPMzVwHRsABcyheVmfdQpxKloKgqh7jjISekYLzTkW6AVKbUCljQ9R7VOKemR6Iv3Bbp_a8Ayk-0SB4QLO_1LGZYNnLXs9DruGeJbAhUbTXFeEk_Ewc_aJlDPlviHF3KhVQ58AGj1p7PPyohoabdlxI_Gv03-kxzavH7VMYsmlvHoa0iHZkLtsgZroKHDyDE63HawWz0uwuqScFjDHF2i-Q6RqYbgdKnrkSJAvHqCuV7H42vWdZYCP-PXehZLsAOheEg5jaxngiw3XbsNVH7R8vUlfvvuhKXRSHVD2_oPD1xNGKx_aal3WGR_20kIMrKPKj4PN-mZk4GzcmoZKb8OXaO0L7C78j6ao6pnJ6oF1-xDBdK_uKvZYaJ_aR-tBbC8ffKYubkKIQ44ddw24bg4-GiwzB-1SKdtAEr9fZFjWo2wmhXzXlz-i6JpUwQqA-TILlISqWQoO4EGTA3Kr1FoDMoIfBfNU-2i8BZ7M0rml_Fe; rsReferrerData=%7B%22currentPageRollup%22%3A%22%2Fjob%2Fjobs-srch%22%2C%22previousPageRollup%22%3A%22%2Fjob%2Fjobs-srch%22%2C%22currentPageAbstract%22%3A%22%2FJob%2F%5BOCC%5D-jobs-SRCH_%5BPRM%5D.htm%22%2C%22previousPageAbstract%22%3A%22%2FJob%2F%5BOCC%5D-jobs-SRCH_%5BPRM%5D.htm%22%2C%22currentPageFull%22%3A%22https%3A%2F%2Fwww.glassdoor.sg%2FJob%2Fsingapore-software-engineer-jobs-SRCH_IL.0%2C9_IC3235921_KO10%2C27.htm%22%2C%22previousPageFull%22%3A%22https%3A%2F%2Fwww.glassdoor.sg%2FJob%2Fsingapore-software-engineer-jobs-SRCH_IL.0%2C9_IC3235921_KO10%2C27.htm%22%7D; bs=ZZrOwH6bDL5phbAIkAt9iw:VOjocgUOtU-VQJL_FkBUajt8f-pynvLg0e_eJFhs2xgrSeYb8sa1w9e2BAG_c4Gmpfy-R1yp02Jg51cVWrYG-H5ebB2wxcc9bPXkj5C1Frk:oZYodGPAXuwXMWxexiFaoSv1EwLWZPUepjBHvL3x6Og; rl_user_id=RudderEncrypt%3AU2FsdGVkX19Tk0I9MKN2jX3V60Q7Qvf8KMe3K7CZ4TI%3D; rl_trait=RudderEncrypt%3AU2FsdGVkX190qk0GQNz7kZQms1G1LkzrtQvaaoDCJRgTVs5LMCCJh6koEbye85pyglYOIe%2FCqAFMDsILJ%2BeWMDrCXtFGyDrsT%2F%2BVjeaHYqgleXGYLJJXG%2B8cFbXlxBOJRK%2BGM9UWLqawDxTGakvRYG7OmYXszZ0HMuUVkBZXeNPnkZ5aBtdXqdg96CYqjijX27XnMMKYZJWbyk%2BzHvi3Zw%3D%3D; rl_group_id=RudderEncrypt%3AU2FsdGVkX18kkxf2040XuZSBtwx%2F5oGSU9LV0EAzfIM%3D; rl_group_trait=RudderEncrypt%3AU2FsdGVkX19YNWVaaI9q6N0LJU12kXUq%2FycS8y%2BYnuQ%3D; _dc_gtm_UA-2595786-1=1; OptanonConsent=isGpcEnabled=0&datestamp=Thu+Nov+30+2023+17%3A00%3A18+GMT%2B0800+(Singapore+Standard+Time)&version=202306.2.0&browserGpcFlag=0&isIABGlobal=false&hosts=&consentId=1b3f41e5-c3c0-418d-9cd9-2902e9bc31d1&interactionCount=1&landingPath=NotLandingPage&groups=C0001%3A1%2CC0003%3A1%2CC0002%3A1%2CC0004%3A1%2CC0017%3A1&AwaitingReconsent=false; rl_anonymous_id=RudderEncrypt%3AU2FsdGVkX1%2FkhWAfBVSOkVuWpWzsSKAJhWNNQeGIrEY9K9QV6TeKPz9USkNBO1bo%2BdKNbNwdc1vNmuOZSOdyOA%3D%3D; rl_session=RudderEncrypt%3AU2FsdGVkX1%2Fd6dWQNy6wGPVaOlDr4vD3v8X3dPyFg0lyISrra89BD2V%2Fju9NG5%2BIHs0lvtakF9Nosms2lOBjgUgGYUcm78aK7wuacxBc2Gq6OWEvZgQHuQzMXhumOaH2HsQF%2BVz8cnvqKF7jicKcPw%3D%3D; _dd_s=rum=2&id=ea847201-e2ff-400d-9cde-fa606d586696&created=1701334687002&expire=1701335749357"""
cookie = http.cookies.SimpleCookie()
cookie.load(cookie_str)
cookie_dict = {k: v.value for k, v in cookie.items()}
print(cookie_dict)

In [ ]:
params=[
    {
        "operationName": "JobSearchResultsQuery",
        "variables": {
            "excludeJobListingIds": [],
            "filterParams": [
                {
                    "filterKey": "fromAge",
                    "values": "1"
                }
            ],
            "keyword": "software engineer",
            "locationId": 3235921,
            "locationType": "CITY",
            "numJobsToShow": 20,
            "originalPageUrl": "https://www.glassdoor.sg/Job/singapore-software-engineer-jobs-SRCH_IL.0,9_IC3235921_KO10,27.htm",
            "parameterUrlInput": "IL.0,9_IC3235921_KO10,27",
            "seoFriendlyUrlInput": "singapore-software-engineer-jobs",
            "seoUrl": True
        },
        "query": "query JobSearchResultsQuery($excludeJobListingIds: [Long!], $keyword: String, $locationId: Int, $locationType: LocationTypeEnum, $numJobsToShow: Int!, $pageCursor: String, $pageNumber: Int, $filterParams: [FilterParams], $originalPageUrl: String, $seoFriendlyUrlInput: String, $parameterUrlInput: String, $seoUrl: Boolean) {\n  jobListings(\n    contextHolder: {searchParams: {excludeJobListingIds: $excludeJobListingIds, keyword: $keyword, locationId: $locationId, locationType: $locationType, numPerPage: $numJobsToShow, pageCursor: $pageCursor, pageNumber: $pageNumber, filterParams: $filterParams, originalPageUrl: $originalPageUrl, seoFriendlyUrlInput: $seoFriendlyUrlInput, parameterUrlInput: $parameterUrlInput, seoUrl: $seoUrl, searchType: SR}}\n  ) {\n    filterOptions\n    indeedCtk\n    jobListings {\n      ...JobView\n      __typename\n    }\n    jobListingSeoLinks {\n      linkItems {\n        position\n        url\n        __typename\n      }\n      __typename\n    }\n    jobSearchTrackingKey\n    jobsPageSeoData {\n      pageMetaDescription\n      pageTitle\n      __typename\n    }\n    paginationCursors {\n      cursor\n      pageNumber\n      __typename\n    }\n    indexablePageForSeo\n    searchResultsMetadata {\n      searchCriteria {\n        implicitLocation {\n          id\n          localizedDisplayName\n          type\n          __typename\n        }\n        keyword\n        location {\n          id\n          shortName\n          localizedShortName\n          localizedDisplayName\n          type\n          __typename\n        }\n        __typename\n      }\n      footerVO {\n        countryMenu {\n          childNavigationLinks {\n            id\n            link\n            textKey\n            __typename\n          }\n          __typename\n        }\n        __typename\n      }\n      helpCenterDomain\n      helpCenterLocale\n      jobAlert {\n        jobAlertExists\n        __typename\n      }\n      jobSerpFaq {\n        questions {\n          answer\n          question\n          __typename\n        }\n        __typename\n      }\n      jobSerpJobOutlook {\n        occupation\n        paragraph\n        __typename\n      }\n      showMachineReadableJobs\n      __typename\n    }\n    serpSeoLinksVO {\n      relatedJobTitlesResults\n      searchedJobTitle\n      searchedKeyword\n      searchedLocationIdAsString\n      searchedLocationSeoName\n      searchedLocationType\n      topCityIdsToNameResults {\n        key\n        value\n        __typename\n      }\n      topEmployerIdsToNameResults {\n        key\n        value\n        __typename\n      }\n      topEmployerNameResults\n      topOccupationResults\n      __typename\n    }\n    totalJobsCount\n    __typename\n  }\n}\n\nfragment JobView on JobListingSearchResult {\n  jobview {\n    header {\n    jobLink\n     adOrderId\n    adOrderSponsorshipLevel\n    advertiserType\n    ageInDays\n    applicationId\n    appliedDate\n    applyUrl\n    applyButtonDisabled\n    blur\n    coverPhoto {\n      url\n      __typename\n    }\n    divisionEmployerName\n    easyApply\n    easyApplyMethod\n    employerNameFromSearch\n    employer {\n      activeStatus\n      bestProfile {\n        id\n        __typename\n      }\n      id\n      name\n      shortName\n      size\n      squareLogoUrl\n      __typename\n    }\n      employerNameFromSearch\n      goc\n      gocConfidence\n      gocId\n      jobCountryId\n      jobLink\n      jobResultTrackingKey\n      jobTitleText\n      locationName\n      needsCommission\n      payCurrency\n      payPeriod\n      payPeriodAdjustedPay {\n        p10\n        p50\n        p90\n        __typename\n      }\n      rating\n      salarySource\n      savedJobId\n      sponsored\n      __typename\n    }\n    job {\n      importConfigId\n      jobTitleId\n      jobTitleText\n      listingId\n      __typename\n    }\n    jobListingAdminDetails {\n      cpcVal\n      importConfigId\n      jobListingId\n      jobSourceId\n      userEligibleForAdminJobDetails\n      __typename\n    }\n    job {\n   description\n   }\n overview {\n      shortName\n      squareLogoUrl\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n"
    }
]

In [ ]:
search_data = []
params=[
    {
        "operationName": "JobSearchResultsQuery",
        "variables": {
            "excludeJobListingIds": [],
            "filterParams": [],
            "keyword": "software engineer",
            "locationId": 3235921,
            "locationType": "CITY",
            "numJobsToShow": 100,
            "originalPageUrl": "https://www.glassdoor.sg/Job/singapore-software-engineer-jobs-SRCH_IL.0,9_IC3235921_KO10,27.htm",
            "parameterUrlInput": "IL.0,9_IC3235921_KO10,27",
            "seoFriendlyUrlInput": "singapore-software-engineer-jobs",
            "seoUrl": True,
        },
        "query": "query JobSearchResultsQuery($excludeJobListingIds: [Long!], $keyword: String, $locationId: Int, $locationType: LocationTypeEnum, $numJobsToShow: Int!, $pageCursor: String, $pageNumber: Int, $filterParams: [FilterParams], $originalPageUrl: String, $seoFriendlyUrlInput: String, $parameterUrlInput: String, $seoUrl: Boolean) {\n  jobListings(\n    contextHolder: {searchParams: {excludeJobListingIds: $excludeJobListingIds, keyword: $keyword, locationId: $locationId, locationType: $locationType, numPerPage: $numJobsToShow, pageCursor: $pageCursor, pageNumber: $pageNumber, filterParams: $filterParams, originalPageUrl: $originalPageUrl, seoFriendlyUrlInput: $seoFriendlyUrlInput, parameterUrlInput: $parameterUrlInput, seoUrl: $seoUrl, searchType: SR}}\n  ) {\n    filterOptions\n    indeedCtk\n    jobListings {\n      ...JobView\n      __typename\n    }\n    jobListingSeoLinks {\n      linkItems {\n        position\n        url\n        __typename\n      }\n      __typename\n    }\n    jobSearchTrackingKey\n    jobsPageSeoData {\n      pageMetaDescription\n      pageTitle\n      __typename\n    }\n    paginationCursors {\n      cursor\n      pageNumber\n      __typename\n    }\n    indexablePageForSeo\n    searchResultsMetadata {\n      searchCriteria {\n        implicitLocation {\n          id\n          localizedDisplayName\n          type\n          __typename\n        }\n        keyword\n        location {\n          id\n          shortName\n          localizedShortName\n          localizedDisplayName\n          type\n          __typename\n        }\n        __typename\n      }\n      footerVO {\n        countryMenu {\n          childNavigationLinks {\n            id\n            link\n            textKey\n            __typename\n          }\n          __typename\n        }\n        __typename\n      }\n      helpCenterDomain\n      helpCenterLocale\n      jobAlert {\n        jobAlertExists\n        __typename\n      }\n      jobSerpFaq {\n        questions {\n          answer\n          question\n          __typename\n        }\n        __typename\n      }\n      jobSerpJobOutlook {\n        occupation\n        paragraph\n        __typename\n      }\n      showMachineReadableJobs\n      __typename\n    }\n    serpSeoLinksVO {\n      relatedJobTitlesResults\n      searchedJobTitle\n      searchedKeyword\n      searchedLocationIdAsString\n      searchedLocationSeoName\n      searchedLocationType\n      topCityIdsToNameResults {\n        key\n        value\n        __typename\n      }\n      topEmployerIdsToNameResults {\n        key\n        value\n        __typename\n      }\n      topEmployerNameResults\n      topOccupationResults\n      __typename\n    }\n    totalJobsCount\n    __typename\n  }\n}\n\nfragment JobView on JobListingSearchResult {\n  jobview {\n    header {\n    jobLink\n     adOrderId\n    adOrderSponsorshipLevel\n    advertiserType\n    ageInDays\n    applicationId\n    appliedDate\n    applyUrl\n    applyButtonDisabled\n    blur\n    coverPhoto {\n      url\n      __typename\n    }\n    divisionEmployerName\n    easyApply\n    easyApplyMethod\n    employerNameFromSearch\n    employer {\n      activeStatus\n      bestProfile {\n        id\n        __typename\n      }\n      id\n      name\n      shortName\n      size\n      squareLogoUrl\n      __typename\n    }\n      employerNameFromSearch\n      goc\n      gocConfidence\n      gocId\n      jobCountryId\n      jobLink\n      jobResultTrackingKey\n      jobTitleText\n      locationName\n      needsCommission\n      payCurrency\n      payPeriod\n      payPeriodAdjustedPay {\n        p10\n        p50\n        p90\n        __typename\n      }\n      rating\n      salarySource\n      savedJobId\n      sponsored\n      __typename\n    }\n    job {\n      importConfigId\n      jobTitleId\n      jobTitleText\n      listingId\n      __typename\n    }\n    jobListingAdminDetails {\n      cpcVal\n      importConfigId\n      jobListingId\n      jobSourceId\n      userEligibleForAdminJobDetails\n      __typename\n    }\n    job {\n   description\n   }\n overview {\n      shortName\n      squareLogoUrl\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n"
    }
]

headers={
    "user-agent": "Mozilla/5.0",
    "gd-csrf-token": 'Fh7m39R-83wE3Ye2T3PKTg:eNfjEnQBnm1MC9J1zNY1E-u141ViCOkdf_EB13F7_lEnvyoQlS4idJkfZ4o1NaBb9PT_Y0eAzJe2sXEriKmoJA:EKvNgRa5ZRFE7Jk1SH_D7Bc-kH2ldUGfezhgBKbDoS0',

}
response = requests.post("https://www.glassdoor.sg/graph",headers=headers,json=params,cookies=cookie_dict)
response.status_code
search_data.append(response.json())

In [ ]:
response.json()[0]['data']['jobListings']['totalJobsCount']//100

In [ ]:
from tqdm import tqdm
count = response.json()[0]['data']['jobListings']['totalJobsCount']//100
for page in tqdm(range(2,count+1)):
    print(page)
    for cursor_obj in response.json()[0]['data']['jobListings']['paginationCursors']:
        if cursor_obj['pageNumber'] == page:
            cursor = cursor_obj['cursor']
            break

    params=[
    {
        "operationName": "JobSearchResultsQuery",
        "variables": {
            "excludeJobListingIds": [],
            "filterParams": [],
            "keyword": "software engineer",
            "locationId": 3235921,
            "locationType": "CITY",
            "numJobsToShow": 100,
            "originalPageUrl": "https://www.glassdoor.sg/Job/singapore-software-engineer-jobs-SRCH_IL.0,9_IC3235921_KO10,27.htm",
            "parameterUrlInput": "IL.0,9_IC3235921_KO10,27",
            "seoFriendlyUrlInput": "singapore-software-engineer-jobs",
            "seoUrl": True,
        },
        "query": "query JobSearchResultsQuery($excludeJobListingIds: [Long!], $keyword: String, $locationId: Int, $locationType: LocationTypeEnum, $numJobsToShow: Int!, $pageCursor: String, $pageNumber: Int, $filterParams: [FilterParams], $originalPageUrl: String, $seoFriendlyUrlInput: String, $parameterUrlInput: String, $seoUrl: Boolean) {\n  jobListings(\n    contextHolder: {searchParams: {excludeJobListingIds: $excludeJobListingIds, keyword: $keyword, locationId: $locationId, locationType: $locationType, numPerPage: $numJobsToShow, pageCursor: $pageCursor, pageNumber: $pageNumber, filterParams: $filterParams, originalPageUrl: $originalPageUrl, seoFriendlyUrlInput: $seoFriendlyUrlInput, parameterUrlInput: $parameterUrlInput, seoUrl: $seoUrl, searchType: SR}}\n  ) {\n    filterOptions\n    indeedCtk\n    jobListings {\n      ...JobView\n      __typename\n    }\n    jobListingSeoLinks {\n      linkItems {\n        position\n        url\n        __typename\n      }\n      __typename\n    }\n    jobSearchTrackingKey\n    jobsPageSeoData {\n      pageMetaDescription\n      pageTitle\n      __typename\n    }\n    paginationCursors {\n      cursor\n      pageNumber\n      __typename\n    }\n    indexablePageForSeo\n    searchResultsMetadata {\n      searchCriteria {\n        implicitLocation {\n          id\n          localizedDisplayName\n          type\n          __typename\n        }\n        keyword\n        location {\n          id\n          shortName\n          localizedShortName\n          localizedDisplayName\n          type\n          __typename\n        }\n        __typename\n      }\n      footerVO {\n        countryMenu {\n          childNavigationLinks {\n            id\n            link\n            textKey\n            __typename\n          }\n          __typename\n        }\n        __typename\n      }\n      helpCenterDomain\n      helpCenterLocale\n      jobAlert {\n        jobAlertExists\n        __typename\n      }\n      jobSerpFaq {\n        questions {\n          answer\n          question\n          __typename\n        }\n        __typename\n      }\n      jobSerpJobOutlook {\n        occupation\n        paragraph\n        __typename\n      }\n      showMachineReadableJobs\n      __typename\n    }\n    serpSeoLinksVO {\n      relatedJobTitlesResults\n      searchedJobTitle\n      searchedKeyword\n      searchedLocationIdAsString\n      searchedLocationSeoName\n      searchedLocationType\n      topCityIdsToNameResults {\n        key\n        value\n        __typename\n      }\n      topEmployerIdsToNameResults {\n        key\n        value\n        __typename\n      }\n      topEmployerNameResults\n      topOccupationResults\n      __typename\n    }\n    totalJobsCount\n    __typename\n  }\n}\n\nfragment JobView on JobListingSearchResult {\n  jobview {\n    header {\n    jobLink\n     adOrderId\n    adOrderSponsorshipLevel\n    advertiserType\n    ageInDays\n    applicationId\n    appliedDate\n    applyUrl\n    applyButtonDisabled\n    blur\n    coverPhoto {\n      url\n      __typename\n    }\n    divisionEmployerName\n    easyApply\n    easyApplyMethod\n    employerNameFromSearch\n    employer {\n      activeStatus\n      bestProfile {\n        id\n        __typename\n      }\n      id\n      name\n      shortName\n      size\n      squareLogoUrl\n      __typename\n    }\n      employerNameFromSearch\n      goc\n      gocConfidence\n      gocId\n      jobCountryId\n      jobLink\n      jobResultTrackingKey\n      jobTitleText\n      locationName\n      needsCommission\n      payCurrency\n      payPeriod\n      payPeriodAdjustedPay {\n        p10\n        p50\n        p90\n        __typename\n      }\n      rating\n      salarySource\n      savedJobId\n      sponsored\n      __typename\n    }\n    job {\n      importConfigId\n      jobTitleId\n      jobTitleText\n      listingId\n      __typename\n    }\n    jobListingAdminDetails {\n      cpcVal\n      importConfigId\n      jobListingId\n      jobSourceId\n      userEligibleForAdminJobDetails\n      __typename\n    }\n    job {\n   description\n   }\n overview {\n      shortName\n      squareLogoUrl\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n"
    }]
    params[0]['variables']['pageNumber'] = page
    params[0]['variables']['pageCursor'] = cursor
    response = requests.post("https://www.glassdoor.sg/graph",headers=headers,json=params,cookies=cookie_dict)
    response.status_code
    search_data.append(response.json())

In [ ]:
datas = []
for data in search_data:
    if data[0] is not None:
        if data[0].get('data',{}).get('jobListings',{}):
            datas.extend(data[0].get('data',{}).get('jobListings',{}).get('jobListings',[]))

In [147]:
datas[0]

{'jobview': {'header': {'jobLink': '/partner/jobListing.htm?pos=101&ao=1110586&s=58&guid=0000018c1ff492a5bd8af8155fdbdd1c&src=GD_JOB_AD&t=SR&vt=w&ea=1&cs=1_81911328&cb=1701343172099&jobListingId=1008727938175&cpc=39721386339D0809&jrtk=5-pdx1-0-1hgfv94nlis9e800-fe860d0bbfab8577---6NYlbfkN0DzjGHh0_vj2MqqQziM9bgaXdjBm_u9pVuj2dwr_YJb93Dk6dbIsmJLleHRve8uhK4ySSLbRM9YCwH06irvn7pr7JZU4fYCUWREH_6f9g47CWjA3EE4Y_Uq61QRzplYv8yH8xjQlAt0J6dWOEJ3MJMnVI4t914vrSGPUW8iSpldCz9SCgMFmLqFD6kphJGpCgd3Mab0vswcvACrAFtbMvKPWhF6IyRVf9-1jCCgS2iHKx2Cr9yQgGiJqtV485bQBnsaOtiLgXMI5thCN4b1jGiuYG8QRRWw-gpWyrLRfLqzyQDVjLuziCsqSJ6ZzX6xQY2VAOgY9GfUOoM2s4aIaTYTv2xhrsbty7f8AHocGCj1gd2D0go2ji6B8qqbrTgZqs08fjIoSmnUaO2d4_Ttl8Ge7zhMZfZSlyZa6u8S3EC2YbAvpsdUaomPxPeY177CBR2BhrRbh6oqHd4qX6AZ3D1LnZ5qY1tSDRhwnaYYfKeo0MtdDknTJAkorKq0MK28OTYA2nACr8GK4MzrysUNOYK885tdscDwCQIhEwdkm5k0twZd1YrbBzJSkNIJDGk-WqXACHP0ep2l-DiEelJ_J13NeNCSt6B3HTqXNP0MGtG4fQvMJlFQJ4nbKxausLf35Fyif3wDB4M560YTkNCi7eBr',
   'adOrderId': 1110586,
   'adOrderSponsorshi

In [159]:
params = [
    {
        "operationName": "JobDetailQuery",
        "variables": {
            "enableReviewSummary": True,
            "jl": 1008957268339,
            "queryString": "pos=201&ao=1136043&s=58&guid=0000018c1f75989b9984b595c768a007&src=GD_JOB_AD&t=SR&vt=w&uido=EF3669F859A459101115EA633EFE0CA1&cs=1_5e41da49&cb=1701334850019&jobListingId=1008999847467&jrtk=5-pdx1-0-1hgfnb67bioeb800-71584843a86c205c",
            "pageTypeEnum": "SERP"
        },
        "query": "query JobDetailQuery($jl: Long!, $queryString: String, $enableReviewSummary: Boolean!, $pageTypeEnum: PageTypeEnum) {\n  jobview: jobView(\n    listingId: $jl\n    contextHolder: {queryString: $queryString, pageTypeEnum: $pageTypeEnum}\n  ) {\n    ...DetailFragment\n    employerReviewSummary @include(if: $enableReviewSummary) {\n      reviewSummary {\n        highlightSummary {\n          sentiment\n          sentence\n          categoryReviewCount\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n}\n\nfragment DetailFragment on JobView {\n  employerBenefits {\n    benefitsOverview {\n      benefitsHighlights {\n        benefit {\n          commentCount\n          icon\n          name\n          __typename\n        }\n        highlightPhrase\n        __typename\n      }\n      overallBenefitRating\n      employerBenefitSummary {\n        comment\n        __typename\n      }\n      __typename\n    }\n    benefitReviews {\n      benefitComments {\n        id\n        comment\n        __typename\n      }\n      cityName\n      createDate\n      currentJob\n      rating\n      stateName\n      userEnteredJobTitle\n      __typename\n    }\n    numReviews\n    __typename\n  }\n  employerContent {\n    featuredVideoLink\n    managedContent {\n      id\n      type\n      title\n      body\n      captions\n      photos\n      videos\n      __typename\n    }\n    diversityContent {\n      goals {\n        id\n        workPopulation\n        underRepresentedGroup\n        currentMetrics\n        currentMetricsDate\n        representationGoalMetrics\n        representationGoalMetricsDate\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n  employerAttributes {\n    attributes {\n      attributeName\n      attributeValue\n      __typename\n    }\n    __typename\n  }\n  gaTrackerData {\n    isSponsoredFromIndeed\n    isSponsoredFromJobListingHit\n    jobViewDisplayTimeMillis\n    requiresTracking\n    pageRequestGuid\n    searchTypeCode\n    trackingUrl\n    __typename\n  }\n  header {\n    jobLink\n    adOrderId\n    adOrderSponsorshipLevel\n    advertiserType\n    ageInDays\n    applicationId\n    appliedDate\n    applyUrl\n    applyButtonDisabled\n    blur\n    coverPhoto {\n      url\n      __typename\n    }\n    divisionEmployerName\n    easyApply\n    easyApplyMethod\n    employerNameFromSearch\n    employer {\n      activeStatus\n      bestProfile {\n        id\n        __typename\n      }\n      id\n      name\n      shortName\n      size\n      squareLogoUrl\n      __typename\n    }\n    expired\n    goc\n    hideCEOInfo\n    indeedApplyMetadata\n    indeedJobAttribute {\n      education\n      skills\n      educationLabel\n      skillsLabel\n      yearsOfExperienceLabel\n      __typename\n    }\n    isIndexableJobViewPage\n    jobTitleText\n    jobType\n    jobTypeKeys\n    jobCountryId\n    jobResultTrackingKey\n    locId\n    locationName\n    locationType\n    needsCommission\n    normalizedJobTitle\n    organic\n    payCurrency\n    payPeriod\n    payPeriodAdjustedPay {\n      p10\n      p50\n      p90\n      __typename\n    }\n    rating\n    remoteWorkTypes\n    salarySource\n    savedJobId\n    seoJobLink\n    sgocId\n    sponsored\n    categoryMgocId\n    urgencySignal {\n      labelKey\n      messageKey\n      normalizedCount\n      __typename\n    }\n    __typename\n  }\n  similarJobs {\n    relatedJobTitle\n    careerUrl\n    __typename\n  }\n  job {\n    description\n    discoverDate\n    eolHashCode\n    importConfigId\n    jobReqId\n    jobSource\n    jobTitleId\n    jobTitleText\n    listingId\n    __typename\n  }\n  jobListingAdminDetails {\n    adOrderId\n    cpcVal\n    importConfigId\n    jobListingId\n    jobSourceId\n    userEligibleForAdminJobDetails\n    __typename\n  }\n  map {\n    address\n    cityName\n    country\n    employer {\n      id\n      name\n      __typename\n    }\n    lat\n    lng\n    locationName\n    postalCode\n    stateName\n    __typename\n  }\n  overview {\n    ceo {\n      name\n      photoUrl\n      __typename\n    }\n    id\n    name\n    shortName\n    squareLogoUrl\n    headquarters\n    links {\n      overviewUrl\n      benefitsUrl\n      photosUrl\n      reviewsUrl\n      salariesUrl\n      __typename\n    }\n    primaryIndustry {\n      industryId\n      industryName\n      sectorName\n      sectorId\n      __typename\n    }\n    ratings {\n      overallRating\n      ceoRating\n      ceoRatingsCount\n      recommendToFriendRating\n      compensationAndBenefitsRating\n      cultureAndValuesRating\n      careerOpportunitiesRating\n      seniorManagementRating\n      workLifeBalanceRating\n      __typename\n    }\n    revenue\n    size\n    sizeCategory\n    type\n    website\n    yearFounded\n    __typename\n  }\n  photos {\n    photos {\n      caption\n      photoId\n      photoId2x\n      photoLink\n      photoUrl\n      photoUrl2x\n      __typename\n    }\n    __typename\n  }\n  reviews {\n    reviews {\n      advice\n      cons\n      countHelpful\n      employerResponses {\n        response\n        responseDateTime\n        userJobTitle\n        __typename\n      }\n      employmentStatus\n      featured\n      isCurrentJob\n      jobTitle {\n        text\n        __typename\n      }\n      lengthOfEmployment\n      pros\n      ratingBusinessOutlook\n      ratingCareerOpportunities\n      ratingCeo\n      ratingCompensationAndBenefits\n      ratingCultureAndValues\n      ratingOverall\n      ratingRecommendToFriend\n      ratingSeniorLeadership\n      ratingWorkLifeBalance\n      reviewDateTime\n      reviewId\n      summary\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n"
    }
]
response = requests.post("https://www.glassdoor.sg/graph",headers=headers,json=params,cookies=cookie_dict)
response.status_code

200

In [ ]:
count=0
job_data = []
import time
for i in tqdm(datas):
    id = i['jobview']['job']['listingId']
    params[0]['variables']['jl'] = id
    response = requests.post("https://www.glassdoor.sg/graph",headers=headers,json=params,cookies=cookie_dict)
    job_data.append(response.json())
    response.status_code
    count+=1
    if count%100==0:
        print("Sleeping")
        time.sleep(20)

In [218]:
from bs4 import BeautifulSoup

In [257]:
pd.DataFrame([job_data[1][0]['data']['jobview']['header']['indeedJobAttribute']])

,education,skills,educationLabel,skillsLabel,yearsOfExperienceLabel,__typename
0,[],[5QH5A],[],[Google Docs],[],IndeedJobAttribute


In [282]:
df=pd.DataFrame()
for i in range(len(job_data)):
    temp = pd.concat([pd.json_normalize(job_data[i][0]['data']['jobview']['job']),pd.json_normalize(job_data[i][0]['data']['jobview']['header']).drop(columns=['jobTitleText']),pd.DataFrame([job_data[i][0]['data']['jobview']['header']['indeedJobAttribute']])],axis=1)[['description', 'discoverDate', 'eolHashCode', 'importConfigId','jobReqId', 'jobSource', 'jobTitleId', 'jobTitleText', 'listingId','ageInDays','jobLink','applyUrl','employerNameFromSearch','expired','goc','jobType','locationName','employer.squareLogoUrl']]
    df = pd.concat([df,temp],axis=0)
df.shape

(1188, 18)

In [283]:
df.reset_index(drop=True).to_json('talenttrove/data/jobs/glassdoor_processed.json')

In [269]:
pd.concat([df,temp],axis=0,ignore_index=True)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [232]:
df.drop_duplicates(subset=['listingId'])

,description,discoverDate,eolHashCode,importConfigId,jobReqId,jobSource,jobTitleId,jobTitleText,jobTitleText,listingId,ageInDays,jobLink,applyUrl,employerNameFromSearch,expired,goc,jobType,locationName
0,<p>Wonderlit is seeking a creative and talente...,2023-11-28T00:00:00,0,322429,1008727938175,Indeed Job Search Platform,0,Creative Writing & English Teacher & Curriculu...,Creative Writing & English Teacher & Curriculu...,1008727938175,154,/partner/jobListing.htm?pos=101&ao=1110586&s=5...,/partner/jobListing.htm?pos=101&ao=1110586&tgt...,Wonderlit Education Centre Pte Ltd,False,english teacher,[Full-time],Orchard
0,<p>OVERALL PURPOSE - Delivering technical and ...,2023-11-03T00:00:00,0,322429,1008738531118,Indeed Job Search Platform,0,Gymnastics Coach (Recreational) / Curriculum D...,Gymnastics Coach (Recreational) / Curriculum D...,1008738531118,149,/partner/jobListing.htm?pos=101&ao=1110586&s=5...,/partner/jobListing.htm?pos=101&ao=1110586&tgt...,Power Kids Gym,False,instructional designer,[Full-time],Singapore
0,<p><b>About us</b></p><p><b>We are a small but...,2023-11-01T00:00:00,0,322429,1008957268339,Indeed Job Search Platform,0,Software Field Engineer,Software Field Engineer,1008957268339,28,/partner/jobListing.htm?pos=101&ao=1110586&s=5...,/partner/jobListing.htm?pos=101&ao=1110586&tgt...,BV-Automation Pte. Ltd.,False,field service engineer,[Full-time],Tampines New Town
0,<p></p><div><p>The CSG Engineering Rotation Pr...,2023-11-29T00:00:00,0,322429,1008950136759,Indeed Job Search Platform,0,Software Engineer - CSG Engineering Rotation P...,Software Engineer - CSG Engineering Rotation P...,1008950136759,32,/partner/jobListing.htm?pos=101&ao=1110586&s=5...,/partner/jobListing.htm?pos=101&ao=1110586&s=5...,Dell Technologies,False,software engineer,None,Singapore
0,<p><b>Job Description:</b></p><ul><li>Design a...,2023-11-17T00:00:00,0,322429,1008986612017,Indeed Job Search Platform,0,Software Engineer (Remote),Software Engineer (Remote),1008986612017,12,/partner/jobListing.htm?pos=101&ao=1110586&s=5...,/partner/jobListing.htm?pos=101&ao=1110586&tgt...,Palazon Technology Pte Ltd,False,software engineer,[Full-time],Remote
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,<div><p><b>Principal Cloud Native Architect</b...,2023-11-03T00:00:00,0,322429,1008904526266,Indeed Job Search Platform,0,Principal Cloud Native Architect,Principal Cloud Native Architect,1008904526266,58,/partner/jobListing.htm?pos=101&ao=1110586&s=5...,/partner/jobListing.htm?pos=101&ao=1110586&s=5...,Dell Technologies,False,cloud engineer,[Full-time],Singapore
0,<div><b>Business Function</b>\n<br>\n<br>\nGro...,2023-11-24T00:00:00,0,322429,1008996303326,Indeed Job Search Platform,0,"Associate, Firewall Engineer, Cloud Engineerin...","Associate, Firewall Engineer, Cloud Engineerin...",1008996303326,5,/partner/jobListing.htm?pos=101&ao=1110586&s=5...,/partner/jobListing.htm?pos=101&ao=1110586&s=5...,DBS Bank Limited,False,cloud engineer,[Full-time],Singapore
0,<div><b>Business Function</b>\n<br>\n<br>\nGro...,2023-11-23T00:00:00,0,322429,1008994575744,Indeed Job Search Platform,0,"Senior Associate, DevOps Engineer, Tech Servic...","Senior Associate, DevOps Engineer, Tech Servic...",1008994575744,7,/partner/jobListing.htm?pos=101&ao=1110586&s=5...,/partner/jobListing.htm?pos=101&ao=1110586&s=5...,DBS Bank Limited,False,devops engineer,[Full-time],Singapore
0,<div><b>Business Function</b>\n<br>\n<br>\nGro...,2023-11-27T00:00:00,0,322429,1008999542936,Indeed Job Search Platform,0,"Senior Associate / Associate, Software Develop...","Senior Associate / Associate, Software Develop...",1008999542936,2,/partner/jobListing.htm?pos=101&ao=1110586&s=5...,/partner/jobListing.htm?pos=101&ao=1110586&s=5...,DBS Bank Limited,False,other,[Full-time],Singapore


In [226]:
df.to_csv('glassdoor_processed.csv',index=False)

,description,discoverDate,eolHashCode,importConfigId,jobReqId,jobSource,jobTitleId,jobTitleText,jobTitleText,listingId,ageInDays,jobLink,applyUrl,employerNameFromSearch,expired,goc,jobType,locationName
0,<p><b>About us</b></p><p><b>We are a small but...,2023-11-01T00:00:00,0,322429,1008957268339,Indeed Job Search Platform,0,Software Field Engineer,Gymnastics Coach (Recreational) / Curriculum D...,1008957268339,149,/partner/jobListing.htm?pos=101&ao=1110586&s=5...,/partner/jobListing.htm?pos=101&ao=1110586&tgt...,Power Kids Gym,False,instructional designer,[Full-time],Singapore
0,<p><b>About us</b></p><p><b>We are a small but...,2023-11-01T00:00:00,0,322429,1008957268339,Indeed Job Search Platform,0,Software Field Engineer,Gymnastics Coach (Recreational) / Curriculum D...,1008957268339,149,/partner/jobListing.htm?pos=101&ao=1110586&s=5...,/partner/jobListing.htm?pos=101&ao=1110586&tgt...,Power Kids Gym,False,instructional designer,[Full-time],Singapore
0,<p><b>About us</b></p><p><b>We are a small but...,2023-11-01T00:00:00,0,322429,1008957268339,Indeed Job Search Platform,0,Software Field Engineer,Gymnastics Coach (Recreational) / Curriculum D...,1008957268339,149,/partner/jobListing.htm?pos=101&ao=1110586&s=5...,/partner/jobListing.htm?pos=101&ao=1110586&tgt...,Power Kids Gym,False,instructional designer,[Full-time],Singapore
0,<p><b>About us</b></p><p><b>We are a small but...,2023-11-01T00:00:00,0,322429,1008957268339,Indeed Job Search Platform,0,Software Field Engineer,Gymnastics Coach (Recreational) / Curriculum D...,1008957268339,149,/partner/jobListing.htm?pos=101&ao=1110586&s=5...,/partner/jobListing.htm?pos=101&ao=1110586&tgt...,Power Kids Gym,False,instructional designer,[Full-time],Singapore
0,<p><b>About us</b></p><p><b>We are a small but...,2023-11-01T00:00:00,0,322429,1008957268339,Indeed Job Search Platform,0,Software Field Engineer,Gymnastics Coach (Recreational) / Curriculum D...,1008957268339,149,/partner/jobListing.htm?pos=101&ao=1110586&s=5...,/partner/jobListing.htm?pos=101&ao=1110586&tgt...,Power Kids Gym,False,instructional designer,[Full-time],Singapore
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,<p><b>About us</b></p><p><b>We are a small but...,2023-11-01T00:00:00,0,322429,1008957268339,Indeed Job Search Platform,0,Software Field Engineer,Gymnastics Coach (Recreational) / Curriculum D...,1008957268339,149,/partner/jobListing.htm?pos=101&ao=1110586&s=5...,/partner/jobListing.htm?pos=101&ao=1110586&tgt...,Power Kids Gym,False,instructional designer,[Full-time],Singapore
0,<p><b>About us</b></p><p><b>We are a small but...,2023-11-01T00:00:00,0,322429,1008957268339,Indeed Job Search Platform,0,Software Field Engineer,Gymnastics Coach (Recreational) / Curriculum D...,1008957268339,149,/partner/jobListing.htm?pos=101&ao=1110586&s=5...,/partner/jobListing.htm?pos=101&ao=1110586&tgt...,Power Kids Gym,False,instructional designer,[Full-time],Singapore
0,<p><b>About us</b></p><p><b>We are a small but...,2023-11-01T00:00:00,0,322429,1008957268339,Indeed Job Search Platform,0,Software Field Engineer,Gymnastics Coach (Recreational) / Curriculum D...,1008957268339,149,/partner/jobListing.htm?pos=101&ao=1110586&s=5...,/partner/jobListing.htm?pos=101&ao=1110586&tgt...,Power Kids Gym,False,instructional designer,[Full-time],Singapore
0,<p><b>About us</b></p><p><b>We are a small but...,2023-11-01T00:00:00,0,322429,1008957268339,Indeed Job Search Platform,0,Software Field Engineer,Gymnastics Coach (Recreational) / Curriculum D...,1008957268339,149,/partner/jobListing.htm?pos=101&ao=1110586&s=5...,/partner/jobListing.htm?pos=101&ao=1110586&tgt...,Power Kids Gym,False,instructional designer,[Full-time],Singapore


## Carbos

In [ ]:
import importlib
import talenttrove.scrapers.carbos
importlib.reload(talenttrove.scrapers.carbos)
from talenttrove.scrapers.carbos import Carbos,ALL_ENGINEERING_GRAD,ALL_ENGINEERING_INTERNSHIPS,ALL_QUANT_TRADING_INTERNSHIPS,ALL_QUANT_TRADING_GRAD

In [ ]:
cookie_str="""__Host-next-auth.csrf-token=198a9ac7d0efdde2761843e0ea10914e75be964787da340ea260f59e878143fd%7Cba8f77fca901b492e38df515db01fb6a544373d922c64a4c602ebb9dc214a723; _ga=GA1.1.81457931.1696941213; __Secure-next-auth.callback-url=https%3A%2F%2Fwww.carbosjobs.com%2F; _ga_JVWQGY8Q4P=GS1.1.1696959406.2.1.1696959543.0.0.0; __Secure-next-auth.session-token=eyJhbGciOiJkaXIiLCJlbmMiOiJBMjU2R0NNIn0..O0NVbTcl0mPJLdu4.Mhtrnhe1gjYLBYzQOEw_CSTUR2qlOTY4ZNupz6V9uxRAQ_GOYYF8CG1-WVNe8vgDTQdKTb9woqFATtlJZ4_XspQHRQJuSjlUZNAu09mlIF9hD1I788ghDLomOqgTyaxbVp5POSDltT-8wPeqe7Y4773wOO4OAoXR5-LBUmUWy2L1g1eG8gJAbkDAp_D0rBBaLzoQk1iNZKMApNWtovxtF6jyw6XNoNOIaokIuPQJAbamD-oB.fI5ORt-lR8ytWMhbi8mFvw; ph_phc_c3EWoh5aCGLJ7aaRb2TkaJOCIGhGJ1AhLMMLR9GJyT8_posthog=%7B%22distinct_id%22%3A%22clnkbhoce0000jw08m32ejv45%22%2C%22%24device_id%22%3A%22018b1994-004d-7b0a-a554-92d0f8c1f935%22%2C%22%24user_state%22%3A%22identified%22%2C%22%24sesid%22%3A%5B1696959551540%2C%22018b1aa9-92eb-7578-aef2-47c19040fd55%22%2C1696959402729%5D%2C%22%24session_recording_enabled_server_side%22%3Atrue%2C%22%24console_log_recording_enabled_server_side%22%3Atrue%2C%22%24session_recording_recorder_version_server_side%22%3A%22v2%22%2C%22%24autocapture_disabled_server_side%22%3Afalse%2C%22%24active_feature_flags%22%3A%5B%5D%2C%22%24enabled_feature_flags%22%3A%7B%7D%2C%22%24feature_flag_payloads%22%3A%7B%7D%2C%22%24user_id%22%3A%22clnkbhoce0000jw08m32ejv45%22%2C%22%24stored_person_properties%22%3A%7B%22email%22%3A%22agarwalpratham2001%40gmail.com%22%2C%22name%22%3A%22Pratham%20Agarwala%22%7D%7D"""
client = Carbos(cookie_str)

In [ ]:
jobs_1 = client.collect(ALL_ENGINEERING_GRAD)
jobs_1 = pd.DataFrame(jobs_1)
jobs_2 = client.collect(ALL_ENGINEERING_INTERNSHIPS)
jobs_2 = pd.DataFrame(jobs_2)
jobs_3 = client.collect(ALL_QUANT_TRADING_INTERNSHIPS)
jobs_3 = pd.DataFrame(jobs_3)
jobs_4 = client.collect(ALL_QUANT_TRADING_GRAD)
jobs_4 = pd.DataFrame(jobs_4)

In [ ]:
pd.concat([jobs_1,jobs_2,jobs_3,jobs_4],axis=0).to_csv("talenttrove/data/jobs/carbos.csv")

## My CarrersFuture

In [ ]:
import importlib
import talenttrove.scrapers.mycareersfuture
importlib.reload(talenttrove.scrapers.mycareersfuture)
from talenttrove.scrapers.mycareersfuture import MyCareersFuture

In [ ]:
client = MyCareersFuture()

In [ ]:
data = client.collect(sleep_time=2)

In [ ]:
from tqdm import tqdm
import time
count=0
for i in tqdm(data[6100:]):
    id = i['uuid']
    response = client.get_job_data(id)
    i['job_data'] = response
    count+=1
    if count%30==0:
        time.sleep(10)


In [ ]:
pd.json_normalize(pd.DataFrame(data)['job_data'],max_level=0).to_json("talenttrove/data/jobs/mycareersfuture_with_descriptions.json",orient="records")

In [ ]:
data = pd.read_json("talenttrove/data/jobs/mycareersfuture.json",orient="records")

## Simplify Company and Job Scraper

In [ ]:
import importlib
import talenttrove.scrapers.simplifyJobs
importlib.reload(talenttrove.scrapers.simplifyJobs)
from talenttrove.scrapers.simplifyJobs import SimplifyJobs

In [ ]:
from talenttrove.scrapers.utils import HEADERS

In [ ]:
response=requests.get(url=f"https://api.simplify.jobs/v2/company?size=100&page=1",headers=HEADERS)

In [ ]:
response

In [ ]:
response.json().get('pages',0)

In [ ]:
client= SimplifyJobs(sleep_time=2)
companies = client.collect()

In [ ]:
pd.json_normalize(companies).to_csv("talenttrove/data/jobs/simplify_companies.csv")

## Techjob Repo Scrapper
- https://techjobrepo.com/

In [ ]:
import importlib
import talenttrove.scrapers.techjobrepo
importlib.reload(talenttrove.scrapers.techjobrepo)
from talenttrove.scrapers.techjobrepo import TechJobRepo

In [ ]:
client= TechJobRepo()
jobs = client.collect()

In [ ]:
import pandas as pd

In [ ]:
pd.json_normalize(jobs).to_csv("talenttrove/data/techjobrepo.csv",index=False)

## DB

In [ ]:
import talenttrove.app.db.sqlite
import importlib
importlib.reload(talenttrove.app.db.sqlite)

In [ ]:
from talenttrove.app.db.sqlite import Database

In [ ]:
db = Database()